### two solvers

Jacobi without weighting often will fail. Gauss-Seidel doesn't have to use weighting.

In [10]:
function Jacobi(A,x,b,p;tol=0.000001)
    D       = diag(A)
    res     = A*x
    resid   = b-res
    tempSol = resid./D
    diff    = sum(resid.^2)
    n    = size(A,1)
    iter = 0
    while ((diff/n > tol) & (iter<1000))
        iter += 1
        x = p*tempSol + (1-p)*x
        res     = A*x
        resid   = b-res
        tempSol = resid./D + x
        diff    = sum(resid.^2)
        println(iter," ",diff/n)
    end
    return x
end

Jacobi (generic function with 1 method)

In [ ]:
function GaussSeidel(A,x,b;tol=0.000001)
    n = size(x,1)
    iter = 0
    diff = 1.0
    while ((diff/n > tol) & (iter<1000))
        iter += 1
        for i=1:n
            x[i] = ((b[i] - A[i,:]*x)/A[i,i])[1,1] + x[i]
        end
        diff = sum((A*x-b).^2)
        println(iter," ",diff/n)
    end
    return x
end

## Two-Way Model with Interaction
### Data

In [1]:
using Distributions
n = 10
pA = 2
pB = 3
A = kron(ones(10),sample(collect(1:pA),n))
B = kron(ones(10),sample(collect(1:pB),n));

In [2]:
strAB = [string(A[i])*"x"*string(B[i]) for i=1:size(A,1)];

levelsmap function in StatsBase works same as mkDict

In [1]:
function mkDict(a) 
    aUnique = unique(a)
    d = Dict()
    for (i,s) in enumerate(aUnique)
        d[s] = i
    end
    return d
end

mkDict (generic function with 1 method)

In [4]:
dA = mkDict(A)
dB = mkDict(B)
dAB = mkDict(strAB);

In [19]:
iA  = int([dA[i] for i in A])
iB  = int([dB[i] for i in B])
iAB = int([dAB[i] for i in strAB]);

In [20]:
ii   = 1:size(A,1)
XA   = sparse(ii,iA,1.0)
XB   = sparse(ii,iB,1.0)
XAB  = sparse(ii,iAB,1.0);

### Computing sparse-stored X'X as product of sparse-stored (X' and X)

In [21]:
nObs = size(A,1)
jj   = fill(1,nObs)
XMu  = sparse(ii,jj,1.0)
@time X = [XMu XA XB XAB];

  0.137627 seconds (86.84 k allocations: 4.342 MB)


In [49]:
@time XPX = X'X
size(XPX)

elapsed time: 0.001881922 seconds (2410240 bytes allocated)


(1479,1479)

#### Simulate y

In [22]:
pA  = size(XA,2)
pB  = size(XB,2)
pAB = size(XAB,2)
α   = randn(pA)
β   = randn(pB)
αβ  = randn(pAB)
y = XA*α + XB*β + XAB*αβ + randn(nObs);

In [24]:
@time XPy = X'y;

elapsed time: 0.013340256 seconds (514260 bytes allocated)


In [42]:
XPXF = full(XPX);

In [46]:
@time XPXi = pinv(XPXF);

elapsed time: 1.123876223 seconds (157718840 bytes allocated, 5.50% gc time)


In [44]:
sol  = XPXi*XPy;

elapsed time: 1.145523672 seconds (157718840 bytes allocated, 3.24% gc time)


In [63]:
p = size(XPX,1)
x = zeros(p)
@time sol = Jacobi(XPX,x,XPy,0.25);

1 205.0883861260506
2 53.18868157148408
3 17.920271047885485
4 7.306110283696749
5 3.3546629393021634
6 1.6528554671956137
7 0.8497389487951501
8 0.4486773407152217
9 0.24109564225493454
10 0.1311063936785637
11 0.07189411891602937
12 0.03966213684710666
13 0.021977288099280193
14 0.012217943197627316
15 0.006809230585457584
16 0.003802061843151804
17 0.002126050120245078
18 0.0011902016715980416
19 0.0006668901447230746
elapsed time: 0.070077784 seconds (1936728 bytes allocated)


In [62]:
[XPX*sol XPy]

1479x2 Array{Float64,2}:
 400.456    400.456  
  81.1796    81.1818 
  48.2585    48.2655 
 -29.2528   -29.2559 
  -8.65785   -8.65344
 -10.6038   -10.6003 
  12.9974    12.9903 
  44.6625    44.6648 
 -55.973    -55.9727 
 103.891    103.893  
  70.2357    70.2343 
  42.3192    42.3218 
 -39.7869   -39.7871 
   ⋮                 
  -7.37793   -7.41238
  15.4255    15.4311 
   4.76078    4.77416
   4.36612    4.36137
  12.6393    12.647  
  30.3358    30.3621 
  18.6257    18.6879 
 -15.5718   -15.5364 
  -8.69301   -8.68792
  -4.94223   -4.95279
   8.84962    8.86273
   4.38614    4.3807 